In [1]:
import cv2
import dlib
import math
import tensorflow as tf
import keras
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from PIL import Image


image_paths = '/content/drive/MyDrive/Spechio_Project/SCUT-FBP5500_v2/Images'


images = []


for file_name in os.listdir(image_paths):
    file_path = os.path.join(image_paths, file_name)


    if os.path.isfile(file_path) and file_name.lower().endswith(('.jpg', '.jpeg', '.png')):

        image = Image.open(file_path)


        images.append((image,file_name))

In [4]:
def compute_hu_moments(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the image to create a binary image
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)


    #contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (assumed to be the face)
    #largest_contour = max(contours, key=cv2.contourArea)

    # Calculate Hu's moments
    moments = cv2.moments(binary)
    hu_moments = cv2.HuMoments(moments)
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments))


    return hu_moments.flatten()


In [5]:
moment_data = []


for image, name in images:
    image_array = np.array(image)

    hu_moments = compute_hu_moments(image_array)
    moment_data.append({'Image Name': name, 'HuM1': hu_moments[0],
                         'HuM2': hu_moments[1], 'HuM3': hu_moments[2],
                         'HuM4': hu_moments[3], 'HuM5': hu_moments[4], 'HuM6': hu_moments[5], 'HuM7': hu_moments[6],})




In [ ]:
moment_data

In [7]:
import pandas as pd
new = pd.DataFrame.from_dict(moment_data)
new

,Image Name,HuM1,HuM2,HuM3,HuM4,HuM5,HuM6,HuM7
0,AM218.jpg,3.008452,7.453705,11.847560,11.061994,-22.518465,-14.789184,-23.571473
1,AM1948.jpg,2.939510,7.042255,12.052818,12.672708,-25.051330,-16.236641,-25.611590
2,AM1946.jpg,2.981613,7.460082,10.863173,10.597546,-21.330205,-14.364314,-22.316672
3,AM214.jpg,2.748384,6.986360,9.662708,9.387926,18.942285,12.945501,19.364469
4,AM1939.jpg,3.042141,8.100464,11.354806,10.838226,-22.403855,-15.233655,21.961341
...,...,...,...,...,...,...,...,...
5496,CF710.jpg,2.818653,6.729160,10.130414,10.211114,-20.431246,-13.592385,-20.727749
5497,CF438.jpg,2.921040,8.002540,11.058992,10.553896,21.362571,14.577149,-22.355600
5498,CF711.jpg,2.993462,7.884668,9.813330,10.603605,-22.012686,-15.676459,20.812936
5499,CF444.jpg,2.783176,7.273966,12.292176,11.326846,23.735805,15.045164,-23.150547


In [8]:
new.isnull().sum()

Image Name    0
HuM1          0
HuM2          0
HuM3          0
HuM4          0
HuM5          0
HuM6          0
HuM7          0
dtype: int64

In [9]:
new.to_csv('/content/drive/MyDrive/hu_moments.csv', index=False)